## Q1. Work on the FIFA 
**dataset: 01-data/02_fifa_Project/FIFA.db**

The FIFA dataset contains seven tables: Attacking, Club, Defending, Goalkeeping, Movement Player and Skill.
The relationship between these seven tables is summarised below:
* pid in Player is also the foreign key to Attacking, Defending, Movement, Goalkeeping,
and Skill.
* club_team_id in Player is the foreign key to Club

In [1]:
import os
import sys
import pandas as pd
import sqlite3
import my_packages.Sql_table_details

# Paths and directory
current_directory = os.getcwd()
fifa = os.path.join(current_directory, "01-data/02_fifa_Project", "fifa.db")

# function that checks table details
my_packages.Sql_table_details.display_table_details(fifa)




Table: Club
+-----+---------------+---------+---------+------------+----+
| cid |      name     |   type  | notnull | dflt_value | pk |
+-----+---------------+---------+---------+------------+----+
|  0  |      cid      | INTEGER |    0    |    None    | 1  |
|  1  |   club_name   |   TEXT  |    0    |    None    | 0  |
|  2  |  league_name  |   TEXT  |    0    |    None    | 0  |
|  3  |  league_level | INTEGER |    0    |    None    | 0  |
|  4  | club_logo_url |   TEXT  |    0    |    None    | 0  |
|  5  | club_flag_url |   TEXT  |    0    |    None    | 0  |
+-----+---------------+---------+---------+------------+----+

Table: Player
+-----+--------------------------+---------+---------+------------+----+
| cid |           name           |   type  | notnull | dflt_value | pk |
+-----+--------------------------+---------+---------+------------+----+
|  0  |           pid            | INTEGER |    0    |    None    | 1  |
|  1  |        short_name        |   TEXT  |    0    |    Non

## 1A. 
Find the average defending rating of players in each club and display the result based
on the average defending rating, ranked from highest to lowest.

In [2]:
defenseQuery = """
    select 
    c.club_name as [Club],
    ROUND(AVG(defending), 2)  as [Avg Defense Rating]
    from Player p
    left join Club c on p.club_team_id=c.cid
    group by 
    c.club_name
    order by avg(p.defending) desc
"""
defenseRate=my_packages.Sql_table_details.run_query_from_db(fifa,defenseQuery)

print(defenseRate)

+-----------------------------------+--------------------+
|                Club               | Avg Defense Rating |
+-----------------------------------+--------------------+
|        Paris Saint-Germain        |       66.07        |
|              Juventus             |       65.87        |
|               Inter               |       65.83        |
|              Atalanta             |       64.77        |
|             Liverpool             |       63.96        |
|            Sporting CP            |       63.33        |
|                Roma               |       63.28        |
|       Clube Atlético Mineiro      |       62.61        |
|              Chelsea              |       62.21        |
|             AS Monaco             |       61.88        |
|             SL Benfica            |       61.85        |
|      Athletic Club de Bilbao      |       61.78        |
|           Leicester City          |       61.62        |
|         Atlético de Madrid        |       61.54       

## 1B.
Find the names of players, their clubs, and their attacking ratings for players who have an attacking rating greater than 80. Note: attacking rating = (crossing + finishing + heading_accuracy + short_passing + volleys)/5

In [3]:
attackQuery = """
    select
    p.short_name as [name], 
    c.club_name as [Club],
    sum(a.crossing + a.finishing + a.heading_accuracy + a.short_passing + a.volleys)/5 as [Attack Rate]

    from Player p
    left join Club c on p.club_team_id=c.cid
    left join Attacking a on p.pid=a.aid    
    group by
    p.short_name,
    c.club_name

    Having
    sum(a.crossing + a.finishing + a.heading_accuracy + a.short_passing + a.volleys)/5 >=80

    order by sum(a.crossing + a.finishing + a.heading_accuracy + a.short_passing + a.volleys)/5 desc
    
"""
AttackRate=my_packages.Sql_table_details.run_query_from_db(fifa,attackQuery)

print(AttackRate)

+-------------------+-------------------------+-------------+
|        name       |           Club          | Attack Rate |
+-------------------+-------------------------+-------------+
|     L. Krejčí     |     AC Sparta Praha     |     126     |
|      R. Riski     |       HJK Helsinki      |     122     |
|      E. Insúa     |  Club Atlético Aldosivi |     119     |
|     O. Popescu    |      FCSB (Steaua)      |     114     |
|    J. González    |       Club Necaxa       |     113     |
|      F. Parra     |       Guaireña FC       |     110     |
|     R. Matsuda    |       Cerezo Osaka      |     108     |
|      A. Rojas     |      Sport Huancayo     |     104     |
|   C. Chukwuemeka  |       Aston Villa       |     104     |
|      L. Rose      |       Macarthur FC      |     101     |
|     Y. Suzuki     |     Shimizu S-Pulse     |     100     |
|    A. Johansson   |       Halmstads BK      |      97     |
|    J. Kitolano    |         Odds BK         |      96     |
|     M.

## 1C.
Find clubs (i.e., the id and the name of the club) where the average shooting rating is
higher than or equal to the average shooting rating of 'AC Milan’.

In [4]:
ACmilanQuery = """
SELECT
    allteams.Club,
    allteams.avg_shooting,
    ac.ACMilan_avg_shooting 
FROM    
(
    SELECT
        1 AS dummykeyAll,
        c.club_name AS Club,
        AVG(p.shooting) AS avg_shooting
    FROM Player p
    LEFT JOIN Club c ON p.club_team_id = c.cid
    LEFT JOIN Attacking a ON p.pid = a.aid   
    GROUP BY
        dummykeyAll,
        c.club_name 
    ORDER BY AVG(p.shooting) DESC
) allteams
INNER JOIN
(
    SELECT
        1 AS dummykeyAC,
        c.club_name AS Club,
        AVG(p.shooting) AS ACMilan_avg_shooting
    FROM Player p
    LEFT JOIN Club c ON p.club_team_id = c.cid
    LEFT JOIN Attacking a ON p.pid = a.aid   
    WHERE c.club_name LIKE '%AC Milan%'
    GROUP BY
        dummykeyAC,
        c.club_name 
) ac ON allteams.dummykeyAll = ac.dummykeyAC

group by
allteams.Club

having
allteams.avg_shooting>= ac.ACMilan_avg_shooting 

order by
allteams.avg_shooting desc
"""

ACmilan = my_packages.Sql_table_details.run_query_from_db(fifa, ACmilanQuery)

print(ACmilan)


+---------------------+--------------------+----------------------+
|         Club        |    avg_shooting    | ACMilan_avg_shooting |
+---------------------+--------------------+----------------------+
|    Real Madrid CF   |        66.5        |        61.88         |
|       Juventus      | 66.17391304347827  |        61.88         |
|  Manchester United  |        66.0        |        61.88         |
|  FC Bayern München  |       65.28        |        61.88         |
|      SL Benfica     | 65.26923076923077  |        61.88         |
|        Inter        | 64.91666666666667  |        61.88         |
|         Ajax        | 64.45833333333333  |        61.88         |
| Paris Saint-Germain | 63.93103448275862  |        61.88         |
|         PSV         |        63.8        |        61.88         |
|      Sevilla FC     | 63.310344827586206 |        61.88         |
|  Atlético de Madrid |       63.25        |        61.88         |
| Real Betis Balompié | 63.06666666666667  |    